In [ ]:
from sqlalchemy import create_engine, text
import sqlalchemy as sa
import pandas as pd
from datetime import datetime

DB_FILE = 'sakila_master.db'

engine = create_engine(f'sqlite:///{DB_FILE}')

print(f'CONNEXION SUCESS: {DB_FILE}')

SQL_QUERY = text("""SELECT * FROM actor;""")
metadata = sa.MetaData()
metadata.reflect(bind=engine)
tables_list = list(metadata.tables.keys())

print(f'TABLES NAMES: {tables_list}')

try:
    with engine.connect() as connexion:
        result = connexion.execute(SQL_QUERY)
        column_names = list(result.keys())
        df = pd.DataFrame(result)

except:
    print('BUG')

CONNEXION SUCESS: sakila_master.db
TABLES NAMES: ['actor', 'address', 'city', 'country', 'category', 'customer', 'store', 'staff', 'film', 'language', 'film_actor', 'film_category', 'film_text', 'inventory', 'payment', 'rental']


In [2]:
df

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2020-12-23 07:12:29
1,2,NICK,WAHLBERG,2020-12-23 07:12:29
2,3,ED,CHASE,2020-12-23 07:12:29
3,4,JENNIFER,DAVIS,2020-12-23 07:12:29
4,5,JOHNNY,LOLLOBRIGIDA,2020-12-23 07:12:29
...,...,...,...,...
195,196,BELA,WALKEN,2020-12-23 07:12:31
196,197,REESE,WEST,2020-12-23 07:12:31
197,198,MARY,KEITEL,2020-12-23 07:12:31
198,199,JULIA,FAWCETT,2020-12-23 07:12:31


## 📝 4 Exercices de SQL Pur

### Exercices de Lecture (SELECT)

1.  **Affichage, Alias, Ordre et Limite :** Affichez les **5 titres de films** (`title` de la table `film`) classés par **ordre alphabétique croissant**, en utilisant un **alias** pour la colonne.
2.  **Agrégation :** Calculez la **moyenne**, le **minimum**, le **maximum**, le **nombre total** et la **somme** du champ `length` (durée en minutes) de tous les films dans la table `film`.

---

### Exercices de Modification (INSERT / UPDATE)

3.  **Insertion (`INSERT`) avec gestion d'ID et de Date :**
    * **Insérez** un nouvel acteur avec le prénom `"JEAN-CLAUDE"`, le nom `"VANDAMME"`, le nouvel ID calculé, et la date/heure actuelle pour le champ **`last_update`**.
      * **Trouvez** l'**`actor_id` maximum** existant dans la table `actor`, puis **calculez le nouvel ID** en ajoutant 1.
      * **Format de date requis pour le `last_update` :** Utilisez le module `datetime` pour générer la chaîne `YYYY-MM-DD HH:MM:SS`.

4.  **Modification (`UPDATE`) par ID :** **Modifiez** l'acteur que vous venez d'insérer (en le ciblant par son **ID unique**) pour changer son prénom de `"JEAN-CLAUDE"` à `"JC"`.

In [3]:
def select_from_db(engine, query) :
    query = text(f"""{query}""")
    try:
        with engine.connect() as connexion:
            result = connexion.execute(query)
            df = pd.DataFrame(result)
            return df
    except :
        print('BUG') 

In [4]:
sql_query = 'SELECT title AS [Top 5 Film alphabetical] FROM film ORDER BY title LIMIT 5;'
select_from_db(engine, sql_query)

,Top 5 Film alphabetical
0,ACADEMY DINOSAUR
1,ACE GOLDFINGER
2,ADAPTATION HOLES
3,AFFAIR PREJUDICE
4,AFRICAN EGG


In [5]:
sql_query = 'SELECT AVG(length), MIN(length), MAX(length), COUNT(length), SUM(length) FROM film'
select_from_db(engine, sql_query)

,AVG(length),MIN(length),MAX(length),COUNT(length),SUM(length)
0,115.272,46,185,1000,115272


In [11]:
return_query = 'SELECT * FROM actor'

max_id = max(select_from_db(engine, return_query).actor_id)
select_from_db(engine, return_query)

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2020-12-23 07:12:29
1,2,NICK,WAHLBERG,2020-12-23 07:12:29
2,3,ED,CHASE,2020-12-23 07:12:29
3,4,JENNIFER,DAVIS,2020-12-23 07:12:29
4,5,JOHNNY,LOLLOBRIGIDA,2020-12-23 07:12:29
...,...,...,...,...
195,196,BELA,WALKEN,2020-12-23 07:12:31
196,197,REESE,WEST,2020-12-23 07:12:31
197,198,MARY,KEITEL,2020-12-23 07:12:31
198,199,JULIA,FAWCETT,2020-12-23 07:12:31


In [7]:
current_time = datetime.now().strftime("%Y-%m-%d %X")

sql_query = f"INSERT INTO actor ('actor_id', 'first_name', 'last_name', 'last_update') VALUES ({max_id+1}, 'JEAN-CLAUDE', 'VANDAMME', '{current_time}');"
insert_query = text(f"""{sql_query}""")

try:
    with engine.connect() as connexion:
        connexion.execute(insert_query)
        connexion.commit()
        df = select_from_db(engine, return_query)
except :
    print('BUG')

In [8]:
sql_query = 'SELECT * FROM actor'
select_from_db(engine, sql_query)

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2020-12-23 07:12:29
1,2,NICK,WAHLBERG,2020-12-23 07:12:29
2,3,ED,CHASE,2020-12-23 07:12:29
3,4,JENNIFER,DAVIS,2020-12-23 07:12:29
4,5,JOHNNY,LOLLOBRIGIDA,2020-12-23 07:12:29
...,...,...,...,...
196,197,REESE,WEST,2020-12-23 07:12:31
197,198,MARY,KEITEL,2020-12-23 07:12:31
198,199,JULIA,FAWCETT,2020-12-23 07:12:31
199,200,THORA,TEMPLE,2020-12-23 07:12:31


In [ ]:
sql_query = f"UPDATE actor SET first_name = 'JC' WHERE actor_id = {max_id+1};"
update_query = text(f"""{sql_query}""")

try:
    with engine.connect() as connexion:
        connexion.execute(update_query)
        connexion.commit()
        df = select_from_db(engine, return_query)
except :
    print('BUG')
df

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2020-12-23 07:12:29
1,2,NICK,WAHLBERG,2020-12-23 07:12:29
2,3,ED,CHASE,2020-12-23 07:12:29
3,4,JENNIFER,DAVIS,2020-12-23 07:12:29
4,5,JOHNNY,LOLLOBRIGIDA,2020-12-23 07:12:29
...,...,...,...,...
196,197,REESE,WEST,2020-12-23 07:12:31
197,198,MARY,KEITEL,2020-12-23 07:12:31
198,199,JULIA,FAWCETT,2020-12-23 07:12:31
199,200,THORA,TEMPLE,2020-12-23 07:12:31


In [ ]:
sql_query = f"DELETE FROM actor WHERE last_name = 'VANDAMME';"
delete_query = text(f"""{sql_query}""")

try:
    with engine.connect() as connexion:
        connexion.execute(delete_query)
        connexion.commit()
        df = select_from_db(engine, return_query)
except :
    print('BUG')
df

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2020-12-23 07:12:29
1,2,NICK,WAHLBERG,2020-12-23 07:12:29
2,3,ED,CHASE,2020-12-23 07:12:29
3,4,JENNIFER,DAVIS,2020-12-23 07:12:29
4,5,JOHNNY,LOLLOBRIGIDA,2020-12-23 07:12:29
...,...,...,...,...
195,196,BELA,WALKEN,2020-12-23 07:12:31
196,197,REESE,WEST,2020-12-23 07:12:31
197,198,MARY,KEITEL,2020-12-23 07:12:31
198,199,JULIA,FAWCETT,2020-12-23 07:12:31
